In [ ]:
# d duncan, dec 6 2018, some code to calculate spatial covariances between wind
#   speeds and SST from observational data
# currently using a high-res gridded SST twice daily product (GHRSST) [that should be closely tied
#  to observed AVHRR from MetOp] and ASCAT 'coastal' 12.5km gridded (swath) winds from MetOp.

# 12/12 -- didn't really get this stuff working... if it's revisited, look at using 'pysal' package for
#  spatial autocorrelation, read up on what other folks have done on SST especially

In [1]:
from netCDF4 import Dataset
import glob


In [2]:
ddir = '/home/dudavid/Dendrite/Dendrite/UserAreas/Dave/covar/'

wfiles = glob.glob(ddir+'OAS*.nc')
sfiles = glob.glob(ddir+'S-OSI*.nc')

day = ['181205']
wfiles = glob.glob(ddir+'OAS*'+day[0]+'*.nc')
sfiles = glob.glob(ddir+'S-OSI*'+day[0]+'*.nc')
print(wfiles)
print(sfiles)

['/home/dudavid/Dendrite/Dendrite/UserAreas/Dave/covar/OASWC12_20181205_082400_62932_M02.nc', '/home/dudavid/Dendrite/Dendrite/UserAreas/Dave/covar/OASWC12_20181205_093000_32246_M01.nc', '/home/dudavid/Dendrite/Dendrite/UserAreas/Dave/covar/OASWC12_20181205_100600_62933_M02.nc', '/home/dudavid/Dendrite/Dendrite/UserAreas/Dave/covar/OASWC12_20181205_111200_32247_M01.nc', '/home/dudavid/Dendrite/Dendrite/UserAreas/Dave/covar/OASWC12_20181205_183300_62938_M02.nc', '/home/dudavid/Dendrite/Dendrite/UserAreas/Dave/covar/OASWC12_20181205_193900_32252_M01.nc', '/home/dudavid/Dendrite/Dendrite/UserAreas/Dave/covar/OASWC12_20181205_211800_32253_M01.nc']
['/home/dudavid/Dendrite/Dendrite/UserAreas/Dave/covar/S-OSI_-FRA_-MTOP-GLBSST_FIELD-201812051200Z.nc']


In [54]:
# to start, pick one of each and find which points correspond with which:
wd = Dataset(wfiles[0])
sd = Dataset(sfiles[0])

ws = wd['wind_speed'][:]
wpos = [ws >= 0]
ws = ws[wpos]
wlo, wla = wd['lon'][:][wpos], wd['lat'][:][wpos]
wlo[wlo>=180] -= 360 # move onto -180 to 180 to match sst grid
# just get rid of values right on meridian?


# find indices on regular .05deg grid to match with closest SST point:
lo_dex = [int(np.mod( np.round( (l-.025+180.0)/0.05),7200 )) for l in wlo] # set any at edge to index 0
la_dex = [int(np.round( (l-.025+ 90.0)/0.05 )) for l in wla]

ss = sd['sea_surface_temperature'][0,:,:] # 3600x7200 grid (.05deg, whole globe, -180to180)
#spos = [ss > 273]
#sst = ss[spos]
slo, sla = sd['lon'][:], sd['lat'][:] # just 1d arrays

slo_like = np.array([slo]*len(sla)) #lazy, but to simplify something below
sla_like = np.array([sla]*len(slo)).transpose()
#slo_like,sla_like = np.indices([len(slo),len(sla)])  # should work, better python anyway
print(np.shape(slo_like),np.shape(sla_like))

#dif = slo[lo_dex]-wlo
#print(dif[abs(dif)>1])
print(info(slo[lo_dex]-wlo)) # good matches should be all smaller than .025deg
print(info(sla[la_dex]-wla))



(3600, 7200) (3600, 7200)
[-179.875 -179.875 -179.875 -179.875 -179.875 -179.875 -179.875 -179.875
 -179.875 -179.875]
[-89.97461  -89.924805 -89.875    -89.825195 -89.77539  -89.72461
 -89.674805 -89.625    -89.575195 -89.52539 ]
['-2.54E-2', '2.54E-2', '-1.25E-5', '1.44E-2']
['-2.54E-2', '2.54E-2', '-6.12E-5', '1.44E-2']


In [64]:

ss_match = ss[la_dex, lo_dex].ravel()
sla_just = sla_like[la_dex, lo_dex].ravel()
slo_just = slo_like[la_dex, lo_dex].ravel()
spos = [ss_match > 273][0]
print(np.shape(ss_match[spos]))


cov1d = np.cov(ss_match[spos],ws[spos]) # '1D' covariance matrix, so global variances and covariance of SST/wind
# [[SST var., covar],[covar, wind var.]]
print('1D covariance matrix: ', cov1d )


(65319,)
1D covariance matrix:  [[ 77.93287942 -10.24587018]
 [-10.24587018  10.4060897 ]]


In [88]:
# now to get some spatial covariance information...

# to calculate weighted correlations:
def cov(x, y, w):
    """Weighted Covariance"""
    return np.sum(w * (x - np.average(x, weights=w)) * (y - np.average(y, weights=w))) / np.sum(w)
    #return np.sum(w * (x - m(x, w)) * (y - m(y, w))) / np.sum(w)

def corr(x, y, w):
    #x = np.array(x)
    #y = np.array(y)
    #w = np.array(w)
    """Weighted Correlation"""
    print('cov internal: ', cov(x,y,w))
    return cov(x, y, w) / np.sqrt(cov(x, x, w) * cov(y, y, w))

#def getDistanceFromLatLonInKm(lat1,lon1,lat2,lon2): 
def getDist(lat1,lon1, lat2,lon2): 
    # Haversine formula (assumes earth a sphere)
    # accepts arrays or single values
    R = 6371 # Radius of the earth in km
    dLat = (lat2-lat1)*np.pi/180  #
    dLon = (lon2-lon1)*np.pi/180 
    a = np.sin(dLat/2) * np.sin(dLat/2) + \
     np.cos(lat1*np.pi/180) * np.cos(lat2*np.pi/180) * np.sin(dLon/2) * np.sin(dLon/2)
    c = 2 * np.arctan2(a**0.5, (1-a)**0.5) 
    d = R * c # Distance in km
    return d

# calculate weighting depending on radial distance from point of interest
la1 = np.array([31,31.1])
la2 = np.array([32,31.2])
lo1 = np.array([30,31.2])
lo2 = np.array([30,31.0])
blah = getDistanceFromLatLonInKm(la1,lo1,la2,lo2)
wgts = blah**(-1) # u
print(blah)
print(wgts)

[111.19492664  24.86394652]
[0.00899322 0.04021888]


In [89]:
# read in wind values, calculate mean spatial correlation coefficient from 
#  points that are within some distance range of the target point
nchoi = 5
chosen = np.random.choice(range(len(ss_match[spos])), nchoi)  # 

for c in chosen:
    clo, cla = slo_just[spos][c], sla_just[spos][c]
    print('random Lo/La: ',clo,cla)
    dists = getDist(cla,clo, sla_just[spos], slo_just[spos] ) #wlo,wla))
    near = np.logical_and(dists < 50, dists > 0) #[dists < 50.0] # limit search radius to X kilometers
    nnear = len(ss_match[spos][near])
    print(nnear)
    #print(info(ss_match[spos][near]))
    wgts = dists[near]**(-1) # weight by inverse distance in km?
    
    print(np.average(np.repeat(ss_match[spos][c],nnear),weights=wgts))
    print(np.repeat(ss_match[spos][c],nnear))
    print(ss_match[spos][near])
    cory = corr( np.repeat(ss_match[spos][c],nnear), ss_match[spos][near], wgts)
    # doesn't work because a repeating array for first arg causes zero in denom and numerator...
    #  how do others do this??
    # can i give it anomalies of SST, so the chosen point is itself anomaly wrt the background scene
    #  of some size? might not work if using same points as in calculation...
    print('weighted corr: ',cory )



random Lo/La:  169.27539 -39.97461
18
285.06
[285.06 285.06 285.06 285.06 285.06 285.06 285.06 285.06 285.06 285.06
 285.06 285.06 285.06 285.06 285.06 285.06 285.06 285.06]
[285.88 284.29999999999995 287.9 285.62 286.56 287.59 285.46999999999997
 288.58 286.77 286.85999999999996 286.26 288.62 288.15999999999997 288.64
 288.13 288.60999999999996 288.96999999999997 284.52]
cov internal:  0.0
weighted corr:  nan
random Lo/La:  -161.6748 6.5253906
24
297.25
[297.25 297.25 297.25 297.25 297.25 297.25 297.25 297.25 297.25 297.25
 297.25 297.25 297.25 297.25 297.25 297.25 297.25 297.25 297.25 297.25
 297.25 297.25 297.25 297.25]
[297.5 302.17999999999995 300.88 297.19 297.60999999999996 302.96 296.65
 302.87 302.88 302.66999999999996 302.5 302.38 300.65999999999997 301.89
 296.96999999999997 302.78 301.08 302.5 298.96999999999997
 302.60999999999996 302.7 300.72999999999996 302.7 297.46]
cov internal:  0.0
weighted corr:  nan
random Lo/La:  -166.3252 37.075195
16
286.60999999999996
[286.61 2

/home/dudavid/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in sqrt
/home/dudavid/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in double_scalars
  from ipykernel import kernelapp as app
